# ML-5 : Time Series Forecasting avec ML.NET

**Navigation** : [Index](README.md) | [<< ML-4-Evaluation](ML-4-Evaluation.ipynb) | [Suivant >>](ML-6-ONNX.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre les principes du forecasting de séries temporelles
2. Utiliser l'algorithme **ForecastBySsa** (Singular Spectrum Analysis)
3. Optimiser les hyperparamètres avec AutoML
4. Evaluer les prévisions avec des métriques adaptées (MAE, RMSE)
5. Visualiser les prédictions avec Plotly

### Prérequis
- ML-1 à ML-4 complétés
- Concepts de base en séries temporelles (tendance, saisonnalité)

### Durée estimée : 45-60 minutes

---

# Forecasting de Séries Temporelles

## Qu'est-ce qu'une série temporelle ?

Une **série temporelle** est une séquence de données points indexés dans le temps. Exemples :
- Ventes mensuelles d'un magasin
- Température journalière
- Cours d'une action en bourse
- Nombre de visiteurs d'un site web

**Composantes d'une série temporelle** :
- **Tendance** : Mouvement général à long terme (hausse/baisse)
- **Saisonnalité** : Pattern qui se répète à intervalles réguliers (ex: ventes de Noël)
- **Cycle** : Pattern non-périodique (ex: cycles économiques)
- **Bruit** : Variabilité aléatoire

## L'algorithme ForecastBySsa

**SSA (Singular Spectrum Analysis)** est une technique de décomposition de séries temporelles qui :

1. Décompose la série en composantes principales (tendance, saisonnalité, bruit)
2. Reconstruit la série pour le forecasting
3. Peut capturer des patterns non-linéaires

**Paramètres clés** :
- `trainSize` : Nombre de points pour l'entraînement
- `seriesLength` : Longueur de la fenêtre d'analyse
- `horizon` : Nombre de pas de temps à prédire

> **Note** : SSA est particulièrement efficace pour les séries avec une forte saisonnalité.

In [ ]:
#r "nuget: Microsoft.ML, 3.0.0"
#r "nuget: Microsoft.ML.TimeSeries, 3.0.0"
#r "nuget: XPlot.Plotly.Interactive, 3.0.2"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.TimeSeries;
using System;
using System.Collections.Generic;
using System.Linq;
using XPlot.Plotly;

Console.WriteLine("Packages chargés avec succès !");

## Exemple 1 : Prévision de ventes mensuelles

Dans ce premier exemple, nous allons prédire les ventes mensuelles d'un magasin.

### Données

Nous avons des données de ventes mensuelles sur 3 ans (36 mois).

In [ ]:
// Définition de la structure de données
public class MonthlySalesData
{
    public float Month { get; set; }    // Numéro du mois (1, 2, 3, ...)
    public float Sales { get; set; }    // Ventes du mois
}

public class SalesForecast
{
    public float[] ForecastedSales { get; set; }
    public float[] ConfidenceLowerBound { get; set; }
    public float[] ConfidenceUpperBound { get; set; }
}

// Créer des données de ventes mensuelles avec tendance + saisonnalité
var salesData = new List<MonthlySalesData>();
var rand = new Random(42);

for (int month = 1; month <= 36; month++)
{
    // Tendance à la hausse
    float trend = 100 + month * 5;
    
    // Saisonnalité (pic en décembre, creux en été)
    float seasonality = (float)(20 * Math.Sin(2 * Math.PI * month / 12));
    
    // Bruit aléatoire
    float noise = (float)((rand.NextDouble() - 0.5) * 10);
    
    float sales = trend + seasonality + noise;
    salesData.Add(new MonthlySalesData { Month = month, Sales = sales });
}

// Afficher les premières lignes
Console.WriteLine("Historique des ventes (6 premiers mois) :");
foreach (var data in salesData.Take(6))
{
    Console.WriteLine($"Mois {data.Month}: {data.Sales:F2} ventes");
}

### Visualisation des données historiques

In [ ]:
var months = salesData.Select(x => (double)x.Month).ToArray();
var sales = salesData.Select(x => (double)x.Sales).ToArray();

var historicalTrace = new Scatter
{
    x = months,
    y = sales,
    mode = "lines+markers",
    name = "Ventes historiques"
};

var layout = new Layout.Layout
{
    title = "Historique des ventes mensuelles",
    xaxis = new Xaxis { title = "Mois" },
    yaxis = new Yaxis { title = "Ventes" }
};

var chart = Chart.Plot(new[] { historicalTrace }, layout);
display(chart);

### Entraînement du modèle de prévision

Nous allons utiliser l'algorithme **ForecastBySsa** pour prédire les 6 prochains mois.

In [ ]:
// Créer le contexte ML
var mlContext = new MLContext(seed: 42);

// Charger les données
var trainingData = mlContext.Data.LoadFromEnumerable(salesData);

// Paramètres du forecasting
int horizon = 6;  // Prédire les 6 prochains mois
int trainSize = 30;  // Utiliser 30 mois pour l'entraînement
int seriesLength = 12;  // Fenêtre d'analyse (12 mois = saisonnalité annuelle)

Console.WriteLine($"Configuration du forecasting:");
Console.WriteLine($"  - Horizon: {horizon} mois");
Console.WriteLine($"  - Train size: {trainSize} mois");
Console.WriteLine($"  - Series length: {seriesLength} mois");

// Créer le pipeline de forecasting
var pipeline = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: "ForecastedSales",
    inputColumnName: "Sales",
    windowSize: seriesLength,
    seriesLength: seriesLength,
    trainSize: trainSize,
    horizon: horizon,
    confidenceLevel: 0.95f
);

// Entraîner le modèle
Console.WriteLine("\nEntraînement du modèle...");
var model = pipeline.Fit(trainingData);
Console.WriteLine("Modèle entraîné !");

### Génération des prévisions

In [ ]:
// Faire des prévisions
var forecastingEngine = mlContext.Model.CreatePredictionEngine<MonthlySalesData, SalesForecast>(model);

// Prédire les 6 prochains mois
var lastMonth = salesData.Last();
var forecast = forecastingEngine.Predict(lastMonth);

Console.WriteLine("\n=== Prévisions pour les 6 prochains mois ===");
for (int i = 0; i < horizon; i++)
{
    int futureMonth = 36 + i + 1;
    float predicted = forecast.ForecastedSales[i];
    float lowerBound = forecast.ConfidenceLowerBound[i];
    float upperBound = forecast.ConfidenceUpperBound[i];
    
    Console.WriteLine($"Mois {futureMonth}: {predicted:F2} ventes [intervalle 95%: {lowerBound:F2} - {upperBound:F2}]");
}

### Visualisation des prévisions

In [ ]:
// Préparer les données pour le graphique
var forecastMonths = Enumerable.Range(37, horizon).Select(x => (double)x).ToArray();

// Trace historique
var historicalTrace = new Scatter
{
    x = months,
    y = sales,
    mode = "lines+markers",
    name = "Historique"
};

// Trace prévision
var forecastTrace = new Scatter
{
    x = forecastMonths,
    y = forecast.ForecastedSales.Select(x => (double)x).ToArray(),
    mode = "lines+markers",
    name = "Prévision"
};

// Trace intervalle de confiance
var upperBoundTrace = new Scatter
{
    x = forecastMonths,
    y = forecast.ConfidenceUpperBound.Select(x => (double)x).ToArray(),
    mode = "lines",
    line = new Line { width = 0 },
    showlegend = false,
    name = "Borne sup"
};

var lowerBoundTrace = new Scatter
{
    x = forecastMonths,
    y = forecast.ConfidenceLowerBound.Select(x => (double)x).ToArray(),
    mode = "lines",
    line = new Line { width = 0 },
    fill = "tonexty",
    fillcolor = "rgba(0,100,80,0.2)",
    showlegend = false,
    name = "Borne inf"
};

var layout = new Layout.Layout
{
    title = "Prévisions des ventes avec intervalle de confiance 95%",
    xaxis = new Xaxis { title = "Mois" },
    yaxis = new Yaxis { title = "Ventes" },
    shapes = new List<Shape>()
};

var chart = Chart.Plot(new[] { historicalTrace, lowerBoundTrace, upperBoundTrace, forecastTrace }, layout);
display(chart);

### Interprétation des résultats

Cette première prévision montre :

| Aspect | Observation | Signification |
|--------|-------------|---------------|
| **Tendance** | Hausse continue | Le modèle capture la tendance à la hausse |
| **Saisonnalité** | Patterns répétitifs | Le modèle détecte la saisonnalité annuelle |
| **Intervalle de confiance** | S'élargit avec le temps | Incertitude croissante pour les prévisions lointaines |

> **Point clé** : L'intervalle de confiance (95%) indique la plage dans laquelle les vraies valeurs ont 95% de chances de se trouver.

## Exemple 2 : Données avec saisonnalité trimestrielle

Dans cet exemple, nous travaillons avec des données financières qui ont une saisonnalité trimestrielle.

In [ ]:
// Données financières (trimestrielles)
var quarterlyData = new List<MonthlySalesData>();
var rand2 = new Random(123);

for (int quarter = 1; quarter <= 20; quarter++)
{
    // Tendance de croissance
    float trend = 1000 + quarter * 50;
    
    // Saisonnalité trimestrielle
    float seasonality = (float)(200 * Math.Sin(2 * Math.PI * quarter / 4));
    
    // Bruit
    float noise = (float)((rand2.NextDouble() - 0.5) * 100);
    
    float value = trend + seasonality + noise;
    quarterlyData.Add(new MonthlySalesData { Month = quarter, Sales = value });
}

// Préparer le forecasting
var quarterlyTrainingData = mlContext.Data.LoadFromEnumerable(quarterlyData);

// Ajuster les paramètres pour la saisonnalité trimestrielle
int quarterlyHorizon = 4;
int quarterlyTrainSize = 16;
int quarterlySeriesLength = 4;  // 4 = saisonnalité trimestrielle

// Pipeline SSA pour données trimestrielles
var quarterlyPipeline = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: "ForecastedSales",
    inputColumnName: "Sales",
    windowSize: quarterlySeriesLength,
    seriesLength: quarterlySeriesLength,
    trainSize: quarterlyTrainSize,
    horizon: quarterlyHorizon,
    confidenceLevel: 0.90f
);

var quarterlyModel = quarterlyPipeline.Fit(quarterlyTrainingData);
var quarterlyEngine = mlContext.Model.CreatePredictionEngine<MonthlySalesData, SalesForecast>(quarterlyModel);

// Prédictions
var lastQuarter = quarterlyData.Last();
var quarterlyForecast = quarterlyEngine.Predict(lastQuarter);

Console.WriteLine("=== Prévisions trimestrielles ===");
for (int i = 0; i < quarterlyHorizon; i++)
{
    int q = 20 + i + 1;
    Console.WriteLine($"Trimestre {q}: {quarterlyForecast.ForecastedSales[i]:F2}");
}

## Exemple 3 : AutoML pour le Time Series Forecasting

ML.NET fournit également **AutoML** pour optimiser automatiquement les hyperparamètres du forecasting.

**Attention** : AutoML pour time series nécessite plus de données et de temps de calcul.

In [ ]:
// Note: AutoML pour Time Series est expérimental et peut nécessiter
// des packages supplémentaires. Voici un exemple conceptuel :

/*
using Microsoft.ML.AutoML;

var experiment = mlContext.Auto()
    .CreateForecastingExperiment()
    .SetDataset(trainingData)
    .SetTimeColumn("Month")
    .SetValueColumn("Sales")
    .SetForecastHorizon(horizon)
    .SetMetric(RegressionMetric.RMSE)
    .SetMaxExperimentTimeInSeconds(60);

var autoMlResult = experiment.Run();

Console.WriteLine($"Meilleur RMSE: {autoMlResult.BestRun.Metric}");
Console.WriteLine($"Meilleur modèle: {autoMlResult.BestRun.TrainerName}");
*/

Console.WriteLine("AutoML pour Time Series : voir documentation ML.NET pour exemples complets");

## Évaluation des prévisions

### Métriques d'évaluation

| Métrique | Formule | Interprétation |
|----------|---------|---------------|
| **MAE** (Mean Absolute Error) | Moyenne(|réel - prédit|) | Erreur moyenne en unités de la variable |
| **RMSE** (Root Mean Squared Error) | √(Moyenne((réel - prédit)²)) | Pénalise les grandes erreurs |
| **MAPE** (Mean Absolute % Error) | Moyenne(|réel - prédit| / réel) × 100 | Erreur en pourcentage |

### Validation du modèle

Pour évaluer correctement un modèle de forecasting :

1. **Hold-out set** : Garder les N dernières données pour validation
2. **Walk-forward validation** : Entraîner sur t=1..N, prédire N+1, puis ré-entraîner
3. **Cross-validation temporelle** : Variante adaptée aux séries temporelles

## Résumé et conclusion

Ce notebook a introduit le **forecasting de séries temporelles** avec ML.NET :

| Concept | Clé |
|---------|-----|
| **SSA (Singular Spectrum Analysis)** | Décompose la série en composantes principales |
| **seriesLength** | Doit correspondre à la saisonnalité (12=annuel, 4=trimestriel) |
| **trainSize** | Nombre de points utilisés pour l'entraînement |
| **horizon** | Nombre de pas de temps à prédire |
| **Intervalle de confiance** | Quantifie l'incertitude des prévisions |

**Points clés** :
1. SSA est efficace pour les séries avec forte saisonnalité
2. L'intervalle de confiance s'élargit avec l'horizon de prévision
3. Le choix de `seriesLength` est critique pour capturer la saisonnalité
4. AutoML peut optimiser les hyperparamètres automatiquement

**Limitations** :
- ForecastBySsa assume une certaine stationnarité de la série
- Les changements brutaux de tendance sont difficiles à prédire
- Nécessite suffisamment de données historiques (minimum 2-3 cycles saisonniers)

**Pour aller plus loin** :
- [ML.NET Forecasting API](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/time-series-serving)
- [CLI AutoML pour Time Series](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/automate-training-with-cli)

---

**Navigation** : [<< ML-4-Evaluation](ML-4-Evaluation.ipynb) | [Suivant >> ML-6-ONNX](ML-6-ONNX.ipynb)